In [2]:
import networkit as nk
from networkit import vizbridges
import pandas as pd
import re
from collections import defaultdict
from ipywidgets import Box
import ast
import numpy as np
import simexpal

In [3]:
# load simex results

allSolutions = {}

def parse(run, f):
    res = {}
    res['status'] = run.get_status().name
    res['experiment'] = run.experiment.name
    res['instance'] = run.instance.shortname
    res['run'] = run
    for var in run.experiment.variation:
        res[var.axis] = var.name
    
    if 'ExactSolution' in run.experiment.name:
        for line in f.readlines():
            if line.startswith('time:'):
                res['time'] = line[7:-1]
            if line.startswith('best edges:'):
                edges = re.findall(r'\(\s?(\d+),\s?(\d+)\)', line)
                res['edges'] = [(int(e[0]), int(e[1])) for e in edges]
        
        if run.experiment.name == 'forestIndexExactSolution':
            solutionsDf = pd.read_csv(run.output_file_path('nkb-FI.csv'))
            solutionsDf.edges = solutionsDf.edges.apply(lambda x: ast.literal_eval(x))
            allSolutions[run.instance.shortname, 'FI'] = solutionsDf
        else:
            solutionsDf = pd.read_csv(run.output_file_path('nkb-THR.csv'))
            solutionsDf.edges = solutionsDf.edges.apply(lambda x: ast.literal_eval(x))
            allSolutions[run.instance.shortname, 'THR'] = solutionsDf
    return res

cfg = simexpal.config_for_dir()
results = []
for run in cfg.discover_all_runs():
    if run.get_status() == simexpal.base.Status.NOT_SUBMITTED: continue
    try:
        with run.open_output_file() as f:
            results.append(parse(run, f))
    except RuntimeError as e:
        pass
        # print('could not open file for run', e, 'run status:', run.get_status())

results = pd.DataFrame(results)
pd.reset_option('display.max_rows')

# dataframe types
results['status'] = results['status'].astype('category')
results['experiment'] = results['experiment'].astype('category')
results['instance'] = results['instance'].astype('category')
results['time'] = pd.to_timedelta(results['time'])

# filter for exact results
results = results.query('experiment == "forestIndexExactSolution" or experiment == "harmonicResistanceExactSolution"')
results

,status,experiment,instance,run,k,edges,time
0,FINISHED,forestIndexExactSolution,ba1,<simexpal.base.Run object at 0x7f2b79d1fd50>,k-5,"[(1, 13), (2, 16), (5, 16), (6, 13), (11, 13)]",0 days 00:22:13.783533
1,FINISHED,forestIndexExactSolution,ba2,<simexpal.base.Run object at 0x7f2b79d1fd10>,k-5,"[(0, 13), (5, 12), (8, 13), (10, 12), (12, 13)]",0 days 00:21:55.635820
2,FINISHED,forestIndexExactSolution,ba3,<simexpal.base.Run object at 0x7f2b79ae0150>,k-5,"[(1, 17), (3, 10), (5, 10), (6, 10), (10, 17)]",0 days 00:22:45.375821
3,FINISHED,forestIndexExactSolution,disturbedRing,<simexpal.base.Run object at 0x7f2b79ae02d0>,k-5,"[(0, 1), (0, 8), (1, 2), (6, 7), (7, 8)]",0 days 00:00:00.103583
4,FINISHED,forestIndexExactSolution,grid5x3,<simexpal.base.Run object at 0x7f2b79ae0390>,k-5,"[(0, 1), (0, 5), (5, 6), (5, 10), (10, 11)]",0 days 00:00:12.223374
5,FINISHED,forestIndexExactSolution,grid5x6,<simexpal.base.Run object at 0x7f2b79ae0790>,k-5,"[(0, 1), (0, 5), (2, 3), (3, 4), (4, 9)]",0 days 00:28:18.629963
6,FINISHED,forestIndexExactSolution,grid7x4,<simexpal.base.Run object at 0x7f2b79ae3650>,k-5,"[(0, 1), (0, 7), (7, 14), (14, 21), (21, 22)]",0 days 00:11:56.150878
7,FINISHED,forestIndexExactSolution,hotdog5x6,<simexpal.base.Run object at 0x7f2b79d1f8d0>,k-5,"[(0, 1), (0, 5), (2, 30), (3, 4), (4, 9)]",0 days 00:41:01.855836
8,FINISHED,forestIndexExactSolution,ws1,<simexpal.base.Run object at 0x7f2b79ae3a50>,k-5,"[(4, 12), (4, 14), (4, 7), (9, 15), (14, 15)]",0 days 00:20:03.899176
9,FINISHED,forestIndexExactSolution,ws2,<simexpal.base.Run object at 0x7f2b79ae3a10>,k-5,"[(2, 13), (5, 13), (8, 14), (12, 14), (13, 14)]",0 days 00:20:27.879030


In [4]:
def plot(row: pd.Series, instance: str, dim: str, k: int, orderColor=False):
    if dim == '2d':
        dimension=nk.vizbridges.Dimension.Two
    else:
        dimension=nk.vizbridges.Dimension.Three
    graph = nk.readGraph(f'../instances/{instance}.nkb')
    graph.indexEdges()
    edgeScores = defaultdict(lambda: 0)
    for i, edge in enumerate(row.edges):
        if i >= k: break
        if orderColor:
            edgeScores[edge] = i
        else:
            edgeScores[edge] = 1
    if row.experiment == 'harmonicResistanceExactSolution':
        color = (0,0,0)
    else:
        color = (0,0,1)
    
    edgePalette = [(0.8,0.8,0.8),color] if not orderColor else None
    widget = nk.vizbridges.widgetFromGraph(graph, dimension=dimension, edgeScores=edgeScores, edgePalette=edgePalette)
    return widget

def plotAllSolutions(df, instance, dim='3d', k=5):
    return Box(tuple(df.query(f'instance == "{instance}"').apply(lambda x: plot(x, instance, dim, k), axis=1)))

def plotResultFromString(graph: str, edges: str, dim):
	if dim == '2d':
		dimension=nk.vizbridges.Dimension.Two
	else:
		dimension=nk.vizbridges.Dimension.Three
	graph = nk.readGraph(f'../instances/{graph}.graph')
	edges = re.findall(r'\(\s?(\d+),\s?(\d+)\)', edges)
	edges = [(int(e[0]), int(e[1])) for e in edges]
	edges = [(e[0], e[1]) if e[0] < e[1] else (e[1],e[0]) for e in edges]
	graph.indexEdges()
	edgeScores = defaultdict(lambda: 0)
	for edge in edges:
		edgeScores[edge] = 1
	return nk.vizbridges.widgetFromGraph(graph, dimension=dimension, edgeScores=edgeScores, edgePalette=[(0.8,0.8,0.8),(0,0,0)])


def heatmapAllSolutions(instance: str, dim: str):
    if dim == '2d':
        dimension=nk.vizbridges.Dimension.Two
    else:
        dimension=nk.vizbridges.Dimension.Three
    FISolutions = allSolutions[instance, 'FI']
    THRSolutions = allSolutions[instance, 'THR']
    graph = nk.readGraph(f'../instances/{instance}.nkb')
    graph.indexEdges()
    FIedgeScores = defaultdict(lambda: 0)
    for row in FISolutions.edges:
        for edge in row:
            FIedgeScores[edge] = FIedgeScores[edge] + 1 
    
    THRedgeScores = defaultdict(lambda: 0)
    for row in THRSolutions.edges:
        for edge in row:
            THRedgeScores[edge] = THRedgeScores[edge] + 1 
    
    FIwidget = nk.vizbridges.widgetFromGraph(graph, dimension=dimension, edgeScores=FIedgeScores)
    THRwidget = nk.vizbridges.widgetFromGraph(graph, dimension=dimension, edgeScores=THRedgeScores)
    print(f'left: forest index ({len(FISolutions)} solutions)\t right: total harmonic resistance({len(THRSolutions)} solutions)')
    return Box((FIwidget, THRwidget))


def drawAllSolutions(instance: str, opt: str, dim: str, centralityMode: str = 'mean', box: bool = True):
    """ opt: FI or THR"""
    if dim == '2d':
        dimension=nk.vizbridges.Dimension.Two
    else:
        dimension=nk.vizbridges.Dimension.Three
    FISolutions = allSolutions[instance, opt]
    graph = nk.readGraph(f'../instances/{instance}.nkb')
    graph.indexEdges()
    bc = nk.centrality.Closeness(graph, False, nk.centrality.ClosenessVariant.STANDARD)
    bc.run()
    nodePercentiles = {}
    prevBcValue = None
    prevNode = None
    numSeen = -1
    numCurrent = 0
    for node, bcValue in reversed(bc.ranking()):
        if prevBcValue and abs(prevBcValue - bcValue) < 1e-6:
            nodePercentiles[node] = nodePercentiles[prevNode]
            numCurrent = numCurrent + 1
        else:
            numSeen = numSeen + numCurrent + 1
            numCurrent = 0
            nodePercentiles[node] = numSeen/graph.numberOfNodes()
            prevBcValue = bcValue
            prevNode = node

    widges = []
    centralities = []
    for row in FISolutions.edges:
        FIedgeScores = defaultdict(lambda: 0)
        edgeCentrality = []
        for edge in row:
            FIedgeScores[edge] = 1
            if centralityMode == 'min':
                edgeCentrality.append(min(nodePercentiles[edge[0]], nodePercentiles[edge[1]]))
            if centralityMode == 'max':
                edgeCentrality.append(max(nodePercentiles[edge[0]], nodePercentiles[edge[1]]))
            if centralityMode == 'mean':
                edgeCentrality.append(np.mean((nodePercentiles[edge[0]], nodePercentiles[edge[1]])))
        edgeCentrality = np.mean(edgeCentrality)
        centralities.append(edgeCentrality)

        FIwidget = nk.vizbridges.widgetFromGraph(graph, dimension=dimension, edgeScores=FIedgeScores, showIds=False)
        widges.append(FIwidget)
    print(centralities)
    if box:
        return Box(widges)
    return widges


def centralityScore(instance: str, opt: str, centralityMode: str = 'mean'):
    """ opt: FI or THR"""
    FISolutions = allSolutions[instance, opt]
    graph = nk.readGraph(f'../instances/{instance}.nkb')
    bc = nk.centrality.Closeness(graph, False, nk.centrality.ClosenessVariant.STANDARD)
    bc.run()
    nodePercentiles = {}
    prevBcValue = None
    prevNode = None
    numSeen = -1
    numCurrent = 0
    for node, bcValue in reversed(bc.ranking()):
        if prevBcValue and abs(prevBcValue - bcValue) < 1e-6:
            nodePercentiles[node] = nodePercentiles[prevNode]
            numCurrent = numCurrent + 1
        else:
            numSeen = numSeen + numCurrent + 1
            numCurrent = 0
            nodePercentiles[node] = numSeen/graph.numberOfNodes()
            prevBcValue = bcValue
            prevNode = node

    centralities = []
    for row in FISolutions.edges:
        edgeCentrality = []
        for edge in row:
            if centralityMode == 'min':
                edgeCentrality.append(min(nodePercentiles[edge[0]], nodePercentiles[edge[1]]))
            if centralityMode == 'max':
                edgeCentrality.append(max(nodePercentiles[edge[0]], nodePercentiles[edge[1]]))
            if centralityMode == 'mean':
                edgeCentrality.append(np.mean((nodePercentiles[edge[0]], nodePercentiles[edge[1]])))
        edgeCentrality = np.mean(edgeCentrality)
        centralities.append(edgeCentrality)

    return np.min(centralities), np.mean(centralities), np.max(centralities)

In [18]:
scores = []

for instance in results.instance.unique():
    for opt in ('FI', 'THR'):
        a,b,c = centralityScore(instance, opt)
        scores.append({
            'instance': instance,
            'min': a,
            'mean': b,
            'max': c,
            'opt': opt
        })

scores = pd.DataFrame(scores).set_index(['instance', 'opt'])
print(scores.query('instance != "disturbedRing"').T.style.format(precision=2).to_latex())
scores.query('instance != "disturbedRing"')

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrr}
instance & \multicolumn{2}{r}{ba1} & \multicolumn{2}{r}{ba2} & \multicolumn{2}{r}{ba3} & \multicolumn{2}{r}{grid5x3} & \multicolumn{2}{r}{grid5x6} & \multicolumn{2}{r}{grid7x4} & \multicolumn{2}{r}{hotdog5x6} & \multicolumn{2}{r}{ws1} & \multicolumn{2}{r}{ws2} & \multicolumn{2}{r}{ws3} \\
opt & FI & THR & FI & THR & FI & THR & FI & THR & FI & THR & FI & THR & FI & THR & FI & THR & FI & THR & FI & THR \\
min & 0.31 & 0.40 & 0.29 & 0.47 & 0.32 & 0.40 & 0.24 & 0.53 & 0.09 & 0.69 & 0.11 & 0.76 & 0.14 & 0.71 & 0.34 & 0.49 & 0.27 & 0.34 & 0.16 & 0.28 \\
mean & 0.33 & 0.40 & 0.29 & 0.47 & 0.34 & 0.40 & 0.24 & 0.60 & 0.10 & 0.69 & 0.11 & 0.76 & 0.14 & 0.71 & 0.34 & 0.49 & 0.27 & 0.34 & 0.16 & 0.28 \\
max & 0.37 & 0.40 & 0.29 & 0.47 & 0.36 & 0.40 & 0.24 & 0.67 & 0.13 & 0.69 & 0.11 & 0.76 & 0.14 & 0.71 & 0.34 & 0.49 & 0.27 & 0.34 & 0.16 & 0.28 \\
\end{tabular}



min      mean       max
instance  opt                              
ba1       FI   0.311111  0.333333  0.366667
          THR  0.400000  0.400000  0.400000
ba2       FI   0.294444  0.294444  0.294444
          THR  0.466667  0.466667  0.466667
ba3       FI   0.322222  0.338889  0.361111
          THR  0.400000  0.400000  0.400000
grid5x3   FI   0.240000  0.240000  0.240000
          THR  0.533333  0.596364  0.666667
grid5x6   FI   0.093333  0.103704  0.133333
          THR  0.693333  0.693333  0.693333
grid7x4   FI   0.114286  0.114286  0.114286
          THR  0.757143  0.757143  0.757143
hotdog5x6 FI   0.143750  0.143750  0.143750
          THR  0.712500  0.712500  0.712500
ws1       FI   0.343750  0.343750  0.343750
          THR  0.487500  0.487500  0.487500
ws2       FI   0.268750  0.268750  0.268750
          THR  0.337500  0.337500  0.337500
ws3       FI   0.156250  0.156250  0.156250
          THR  0.275000  0.275000  0.275000

In [8]:
drawAllSolutions('grid5x3', 'THR', '2d', 'mean', box=False)[2] # 0.53

[0.5866666666666667, 0.6666666666666667, 0.5333333333333333, 0.5333333333333333, 0.6666666666666666, 0.5333333333333333, 0.5866666666666667, 0.5333333333333333, 0.6666666666666666, 0.6666666666666666, 0.5866666666666667]


CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [9]:
drawAllSolutions('grid5x3', 'FI', '2d', 'mean', box=False)[1]

[0.24, 0.24]


CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [10]:
drawAllSolutions('grid5x6', 'FI', '2d', 'mean', box=False)[0]

[0.13333333333333333, 0.10666666666666666, 0.10666666666666666, 0.09333333333333334, 0.09333333333333334, 0.09333333333333334, 0.09333333333333334, 0.10666666666666666, 0.10666666666666666, 0.10666666666666666, 0.10666666666666666, 0.09333333333333334, 0.09333333333333334, 0.09333333333333334, 0.09333333333333334, 0.09333333333333334, 0.10666666666666666, 0.10666666666666666, 0.10666666666666666, 0.10666666666666666, 0.10666666666666666, 0.09333333333333334, 0.09333333333333334, 0.09333333333333334, 0.10666666666666666, 0.13333333333333333, 0.13333333333333333]


CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [11]:
drawAllSolutions('grid5x6', 'THR', '2d', 'mean')

[0.6933333333333334]


Box(children=(CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': …

In [13]:
drawAllSolutions('hotdog5x6', 'THR', '2d', 'mean')

[0.7125]


Box(children=(CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': …

In [15]:
drawAllSolutions('hotdog5x6', 'FI', '2d', 'mean')

[0.14375, 0.14375]


Box(children=(CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': …

In [16]:
drawAllSolutions('grid7x4', 'FI', '2d', 'mean')

[0.11428571428571428, 0.11428571428571428]


Box(children=(CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': …

In [18]:
drawAllSolutions('grid7x4', 'THR', '2d', 'mean')

[0.7571428571428571, 0.7571428571428571]


Box(children=(CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': …

In [5]:
heatmapAllSolutions('ba1', '2d')

left: forest index (3 solutions)	 right: total harmonic resistance(1 solutions)


Box(children=(CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': …

In [7]:
heatmapAllSolutions('ws1', '2d')

left: forest index (1 solutions)	 right: total harmonic resistance(1 solutions)


Box(children=(CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': …